In [3]:
%matplotlib inline

Note: if plots don't show, there is a recent matplotlib issue that may be related:
https://github.com/matplotlib/matplotlib/issues/18396


# Implement a Denoising Autoencoder


Use soundpy to implement a denoising autoencoder to clean noisy speech files.

To see how soundpy implements this, see `soundpy.builtin.denoiser_run`.

In [2]:
# to be able to import soundpy from parent directory:
import os
package_dir = '../'
os.chdir(package_dir)

Let's import soundpy, assuming it is in your working directory:



In [4]:
import soundpy as sp;
import IPython.display as ipd
from soundpy import models as spdl

## Prepare for Implementation: Data Organization

### Set model pathway


Currently, this expects a model saved with weights, with a .h5 extension.
(See `model` below)



**soundpy offers a pre-trained denoiser.** 

*(Scroll to bottom for more info on what the denoiser was trained on)*

In [5]:
# Use function 'string2pathlib' to turn string path into pathlib object 
# This allows flexibility across operating systems
model = sp.string2pathlib('audiodata/models/denoiser/example_denoiser_stft.h5')
# as pathlib object, can easily navigate pathway:
model_dir = model.parent

What is in this folder?



In [6]:
files = list(model_dir.glob('*.*'))
for f in files:
    print(f.name)

example_denoiser_stft.h5
log_extraction_settings.csv
log.csv


### Load feature extraction settings

For implementation, it is important that *the same features are extracted from new data as those the model was trained on.*

You can save how features were extracted different ways.

If soundpy extracts features for you, a 'log_extraction_settings.csv' 
file will be saved, which includes relevant feature settings for implementing 
the model. See soundpy.feats.save_features_datasets

In [7]:
feat_settings = sp.utils.load_dict(
    model_dir.joinpath('log_extraction_settings.csv'))
for key, value in feat_settings.items():
    print(key, ' --> ', value)
    # change objects that were string to original format
    import ast
    try:
        feat_settings[key] = ast.literal_eval(value)
    except ValueError:
        pass
    except SyntaxError:
        pass

dur_sec  -->  3
feature_type  -->  stft noisy
feat_type  -->  stft
complex_vals  -->  True
sr  -->  22050
num_feats  -->  177
n_fft  -->  352
win_size_ms  -->  16
frame_length  -->  352
percent_overlap  -->  0.5
window  -->  hann
frames_per_sample  -->  11
labeled_data  -->  False
visualize  -->  True
input_shape  -->  (35, 11, 177)
desired_shape  -->  (385, 177)
use_librosa  -->  True
center  -->  True
mode  -->  reflect
subsection_data  -->  True
divide_factor  -->  10


### For the purposes of plotting, let's use some of the settings defined:



In [8]:
feature_type = feat_settings['feat_type']
sr = feat_settings['sr']

## Provide a new audiofile for the denoiser to denoise!

### We'll use a speech signal provided by soundpy and add white noise to it

In [9]:
speech = sp.string2pathlib('audiodata/python.wav')
s, sr = sp.loadsound(speech, sr=sr)
ipd.Audio(s,rate=sr)

In [10]:
sp.plotsound(s, sr=sr, feature_type='signal', title='Speech (signal)')

In [11]:
sp.plotsound(s, sr=sr, feature_type=feature_type, title='Speech ({})'.format(feature_type.upper()))

## Add white noise: stationary noise

In [12]:
snr = 10

In [13]:
s_wn = sp.augment.add_white_noise(s, sr=sr, snr=snr)

/home/airos/Projects/github/a-n-rose/Python-Sound-Tool/soundpy/feats.py:867: UserWarning: 
  warnings.warn(msg)
/home/airos/Projects/github/a-n-rose/Python-Sound-Tool/soundpy/dsp.py:2769: UserWarning: 
  warnings.warn(msg)
/home/airos/Projects/github/a-n-rose/Python-Sound-Tool/soundpy/dsp.py:756: UserWarning: 

  warnings.warn(msg)


In [14]:
ipd.Audio(s_wn,rate=sr)

In [15]:
sp.plotsound(s_wn, sr=sr, feature_type=feature_type, title='Speech ({})\nwith white noise {} SNR'.format(feature_type.upper(), snr))

# Built-In Denoiser Functionality

We just need to feed the model path, the noisy sample path, and 
the feature settings dictionary we looked at above.



In [16]:
y, sr = spdl.denoiser_run(model, s_wn, feat_settings)

/home/airos/Projects/github/a-n-rose/Python-Sound-Tool/soundpy/models/builtin.py:758: UserWarning: 
  warnings.warn(msg)


## How does the denoised speech sound?


In [17]:
ipd.Audio(y,rate=sr)

## Compare with Wiener Filter

In [18]:
s_wf, sr = sp.filtersignal(s_wn, sr=sr)
ipd.Audio(s_wf, rate=sr)

/home/airos/Projects/github/a-n-rose/Python-Sound-Tool/soundpy/builtin.py:138: UserWarning: 

If possible, perhaps use 8000, 16000, 41000, or 48000 sample rate instead.

  warnings.warn('\n\nWARNING: sample rate 22050 may have some '+\


## Compared with the noisy signal

In [19]:
ipd.Audio(s_wn,rate=sr)

## Compared with the clean signal

In [20]:
ipd.Audio(s,rate=sr)

## How does is the denoised speech look? 



In [21]:
sp.plotsound(y, sr=sr, feature_type = 'signal')

### STFT features of the denoised speech



In [22]:
sp.plotsound(y, sr=sr, feature_type = 'stft', energy_scale = 'power_to_db',
               title = 'Denoiser Output: STFT features')

### STFT features of the Wiener filtered version of the audio:

In [23]:
sp.plotsound(s_wf, sr=sr, feature_type = 'stft', energy_scale = 'power_to_db',
               title = 'Wiener filtered audiofile: STFT features')

### STFT features of the noisy version of the audio:



In [24]:
sp.plotsound(s_wn, sr=sr, feature_type = 'stft', energy_scale = 'power_to_db',
               title = 'Noisy input audiofile: STFT features')

### STFT features of the clean version of the audio:



In [25]:
sp.plotsound(s, sr=sr, feature_type = 'stft', energy_scale = 'power_to_db',
               title = 'Clean "target" audiofile: STFT features')

# Try with non-stationary noise: car horn!

In [26]:
horn = sp.string2pathlib('audiodata/car_horn.wav')
h, sr = sp.loadsound(horn, sr=sr)

In [27]:
ipd.Audio(h,rate=sr)

# Add the car horn to speech: 20 SNR

In [28]:
s_h, snr = sp.dsp.add_backgroundsound(s, h, sr=sr, snr = 20, total_len_sec=3, stationary_noise = False, wrap = True)

/home/airos/Projects/github/a-n-rose/Python-Sound-Tool/soundpy/feats.py:867: UserWarning: 
  warnings.warn(msg)
/home/airos/Projects/github/a-n-rose/Python-Sound-Tool/soundpy/dsp.py:2769: UserWarning: 
  warnings.warn(msg)
/home/airos/Projects/github/a-n-rose/Python-Sound-Tool/soundpy/dsp.py:756: UserWarning: 

  warnings.warn(msg)


In [29]:
ipd.Audio(s_h,rate=sr)

## Apply the Denoiser

In [30]:
y_h, sr = spdl.denoiser_run(model, s_h, feat_settings)

/home/airos/Projects/github/a-n-rose/Python-Sound-Tool/soundpy/models/builtin.py:758: UserWarning: 
  warnings.warn(msg)


In [31]:
ipd.Audio(y_h,rate=sr)

## Compare with Wiener Filter

In [32]:
y_h_wf, sr = sp.filtersignal(y_h, sr=sr)

/home/airos/Projects/github/a-n-rose/Python-Sound-Tool/soundpy/builtin.py:138: UserWarning: 

If possible, perhaps use 8000, 16000, 41000, or 48000 sample rate instead.

  warnings.warn('\n\nWARNING: sample rate 22050 may have some '+\


In [33]:
ipd.Audio(y_h_wf,rate=sr)

# Add the car horn to speech: 10 SNR

In [34]:
s_h_10, snr = sp.dsp.add_backgroundsound(s, h, sr=sr, snr = 10, total_len_sec=3, stationary_noise = False, wrap = True)

In [35]:
ipd.Audio(s_h_10,rate=sr)

## Apply the denoiser

In [36]:
y_h_10_denoiser, sr = spdl.denoiser_run(model, s_h_10, feat_settings)

In [37]:
ipd.Audio(y_h_10_denoiser,rate=sr)

## Compare with Wiener Filter

In [38]:
y_h_wf, sr = sp.filtersignal(s_h_10, sr=sr)

In [39]:
ipd.Audio(y_h_wf,rate=sr)

# In sum

The denoiser did pretty well with stationary noise. It was not trained much on non-stationary noise, such as a car honk. **Might it perform better if so?**

Note: the model was trained on <a href='https://depts.washington.edu/phonlab/resources/uwnu/uwnu1/'>speech</a> corrupted by the following three background sounds, at various snr levels:

In [40]:
traffic = sp.string2pathlib('audiodata/background_samples/traffic.wav')
t, sr = sp.loadsound(traffic, sr=sr)
ipd.Audio(t,rate=sr)

In [41]:
fridge = sp.string2pathlib('audiodata/background_samples/fridge.wav')
f, sr = sp.loadsound(fridge, sr=sr)
ipd.Audio(f,rate=sr)

In [42]:
cafe = sp.string2pathlib('audiodata/background_samples/cafe.wav')
c, sr = sp.loadsound(cafe, sr=sr)
ipd.Audio(c,rate=sr)